In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.491, 0.482, 0.446],
            std= [0.247, 0.243, 0.261]
        )]) # TODO: Automate calculation given a dataset
        
trainset = torchvision.datasets.CIFAR10(root='./raid/data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./raid/data', train=False,
                                       download=True, transform=transform)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

0.0%

100.0%

Extracting ./raid/data/cifar-10-python.tar.gz to ./raid/data
Files already downloaded and verified


In [3]:
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

In [4]:
import gensim.downloader
import numpy as np

model = 'glove-wiki-gigaword-50'
word_vectors = gensim.downloader.load(model)

def transform_targets(x):
    return word_vectors[idx_to_class[x]]

for s in [trainset, testset]:
    idx_to_class = {y:x for x,y in s.class_to_idx.items()}
    s.targets = np.array(list(map(transform_targets, s.targets)))

In [5]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

In [6]:
from torchvision import models
import torch.nn as nn
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(in_features=512, out_features=50)

In [7]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)

In [ ]:
model.train()
for epoch in range(15):  # loop over the dataset multiple times
    print("########## {} ##########".format(epoch+1))
    train_loss = 0.0
    total  = 0
    correct = 0
    for i, data in enumerate(trainloader, 0):

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        if torch.cuda.is_available():
            model.cuda()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        train_loss += loss.item()
        total += labels.size(0)
        labels, outputs = labels.to("cpu"), outputs.to("cpu")
        for l, o in zip(labels, outputs):
            label_word = word_vectors.similar_by_vector(l.numpy(), topn=1)
            output_word = word_vectors.similar_by_vector(o.data.numpy(), topn=1)
            correct += label_word[0][0] == output_word[0][0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print("Loss: {} | Acc: {} | {}/{}".format(train_loss/200, 100.*correct/total, correct, total))
            train_loss = 0
            print(label_word, output_word)

print('Finished Training')

########## 1 ##########
Loss: 2.2050839708931744 | Acc: 36.15 | 2892/8000
[('dog', 1.0)] [('dog', 0.8927386999130249)]
Loss: 1.5867854505218566 | Acc: 45.6125 | 7298/16000
[('automobile', 1.0)] [('automobile', 0.9108813405036926)]


In [ ]:
torch.save(model.state_dict(), "./raid/trained_models/pred_emb.pt")